In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # Generate y = Xw + b + noise.

"""读取数据集"""
batch_size=10
data_iter=d2l.load_array((features,labels),batch_size)

# 读取并打印第一小批量样本
next(iter(data_iter))

"""定义模型"""


'定义模型'

我们首先定义一个模型变量net，它是一个sequential类的实例。Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

在pytorch中，全连接层在Linear类中定义。值得注意的是，我们将两个参数传递到nn.Linear中。第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1.

In [2]:
# nn是神经网络的缩写
from torch import nn

net=nn.Sequential(nn.Linear(2,1))

In [3]:

"""初始化模型参数"""
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

"""定义损失函数"""
# MSELoss类
loss=nn.MSELoss() # 均方误差，默认情况下，它返回所有样本损失的平均值。

"""定义优化算法"""
# SGD:stochastic gradient descent 随机梯度下降
trainer=torch.optim.SGD(net.parameters(),lr=0.03)
# 当我们实例化一个SGD实例时，
# 我们要指定优化的参数 
# （可通过net.parameters()从我们的模型中获得）
# 以及优化算法所需的超参数字典。 
# 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

"""训练"""
num_epochs=3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l=loss(net(X),y) # 通过调用net(X)生成预测并计算损失l(前向传播)
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step() # 使用的是参数空间对应的梯度
    l=loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch 1, loss 0.000395
epoch 2, loss 0.000098
epoch 3, loss 0.000099
w的估计误差： tensor([-5.9366e-05, -3.6240e-04])
b的估计误差： tensor([0.0011])
